In [1]:
# Importing the required libraries

In [2]:
import torch
import torchvision
import numpy as np
from torchvision import transforms
from PIL import Image
from os import listdir
import random
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import time


In [3]:
# Normalization

In [4]:
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    normalize])

In [5]:
# Loading the dataset

In [6]:
train_data_batch = []
target_list = []
train_data = []
batchSize = 10

files = listdir('images/train')
for i in range(len(listdir('images/train'))):
    f = random.choice(files)
    files.remove(f)
    img = Image.open("images/train/" + f)

    img_tensor = transform(img)
    
    
    train_data_batch.append(img_tensor)
    #cracks = 1 if 'cracks' in f else 0  -->>> errado
    nocracks = 1 if 'nocracks' in f else 0
    cracks = 1 - nocracks
    
    target = [cracks, nocracks]
    target_list.append(target)

    
    if len(train_data_batch) >= batchSize: # 10 = tamanho do batch
        train_data.append((train_data_batch, target_list))
       
        train_data_batch = []
        target_list = []
        
       # print('Loaded batch', len(train_data_list), 'of', int(len('images/train'))/2)
       # print('Percentage Done: ', len (train_data)/int(len(listdir('images/train'))/2), '%')
        
        
    

In [7]:
# Neural Feed Forward

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=3, stride = 1, padding=1)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=3, stride = 1, padding=1)
        self.conv3 = nn.Conv2d(12, 18, kernel_size=3, stride = 1, padding=1)
        self.conv4 = nn.Conv2d(18, 24, kernel_size=3, stride = 1, padding=1)
        self.fc1 = nn.Linear(6144, 60) #in and out dataset
        self.fc2 = nn.Linear(60, 2) ##### 10 --> 2
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        
        x = self.conv3(x)
        x =F.max_pool2d(x, 2)
        x = F.relu(x)
        
        x = self.conv4(x)
        x =F.max_pool2d(x, 2)
        x = F.relu(x)
        
        x = x.view(x.size(0), -1)
       
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.sigmoid(x)
        
                

In [9]:
# Train

In [10]:
model = Net()
model.train()
#model.cuda()

optimizer = optim.Adam(model.parameters(), lr=0.01)
def train(epoch):
    
    batch_id = 0
    correct=0
    for data, target in train_data:
        #data = data.cuda()
        data = pad_sequence(data, batch_first=True)
        
        target = torch.Tensor(target) #.cuda()
        
        #print("T=",target.shape)
        data = Variable(data)
        target = Variable(target)
        optimizer.zero_grad()
        out = model(data)
        
        _, predicted = torch.max(out.data, 1)
        _, t = torch.max(target, 1)
        
        correct += (predicted.cpu() == t.cpu()).sum().numpy()
        
        
        criterion = F.binary_cross_entropy
       
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        batch_id = batch_id + 1
        print('Train Epoch: {} [{}/{} ({:.0f}%)] \tLoss; {:.6f}; \tacurracy {:.2f}'.format(epoch, batch_id * len(data), len(train_data)*len(data), 100. * batch_id / len(train_data), loss.item(), correct/(batch_id*len(data))))
        


In [11]:
# Test

In [12]:
def test(): 
    model.eval()
    files = listdir('images/test/')
    f = random.choice(files)
    img = Image.open("images/test/" + f)
    img_eval_tensor = transform(img)
    img_eval_tensor = img_eval_tensor.unsqueeze(0)
    data = Variable(img_eval_tensor) #img_eval_tensor.cuda()
    
    out = model(data)
    print(out.data.max(1, keepdim=True)[1])
    img.show()
    time.sleep(5)
   
    

for epoch in range(1, 30):
    train(epoch)
    test()




Train Epoch: 1 [10/40 (25%)] 	Loss; 0.682940; 	acurracy 0.70
Train Epoch: 1 [20/40 (50%)] 	Loss; 2.210621; 	acurracy 0.55
Train Epoch: 1 [30/40 (75%)] 	Loss; 0.679486; 	acurracy 0.57
Train Epoch: 1 [40/40 (100%)] 	Loss; 0.655552; 	acurracy 0.60
tensor([[ 1]])
Train Epoch: 2 [10/40 (25%)] 	Loss; 0.768606; 	acurracy 0.30
Train Epoch: 2 [20/40 (50%)] 	Loss; 0.677431; 	acurracy 0.45
Train Epoch: 2 [30/40 (75%)] 	Loss; 0.712428; 	acurracy 0.43
Train Epoch: 2 [40/40 (100%)] 	Loss; 0.684728; 	acurracy 0.50
tensor([[ 0]])
Train Epoch: 3 [10/40 (25%)] 	Loss; 0.694451; 	acurracy 0.40
Train Epoch: 3 [20/40 (50%)] 	Loss; 0.690723; 	acurracy 0.40
Train Epoch: 3 [30/40 (75%)] 	Loss; 0.660670; 	acurracy 0.47
Train Epoch: 3 [40/40 (100%)] 	Loss; 0.724251; 	acurracy 0.42
tensor([[ 0]])
Train Epoch: 4 [10/40 (25%)] 	Loss; 0.620487; 	acurracy 0.70
Train Epoch: 4 [20/40 (50%)] 	Loss; 0.638109; 	acurracy 0.55
Train Epoch: 4 [30/40 (75%)] 	Loss; 0.501356; 	acurracy 0.63
Train Epoch: 4 [40/40 (100%)] 	Loss; 